# This notebook compares ReAct with One Big Prompt

there are three variations to test
* one big prompt
* ReAct using RAG as the search function
* ReAct with search function that returns the whole PDF (Part.from_data('pdf'))

In [11]:
import sys
import os
import importlib
import json
from vertexai.generative_models import GenerationConfig, GenerativeModel, Content, Part  
sys.path.append(os.path.abspath('../utils'))
from rich.markdown import Markdown as rich_Markdown

# convert tool
import tool_functions
importlib.reload(tool_functions)
from tool_functions import convert_to_tool # convert self-defined functions to Tool objects

# RAG class
import rag
importlib.reload(rag)
from rag import RAG # RAG search function

# doc_ai chunks
import doc_ai
importlib.reload(doc_ai)
from doc_ai import read_jsonl_to_json_list

# ReAct Agent
import react_agent
importlib.reload(react_agent)
from react_agent import ReactAgent

# one big prompt Agent
import big_prompt_agent
importlib.reload(big_prompt_agent)
from big_prompt_agent import BigPromptAgent

# naf23.pdf

In [2]:
pdf_path='../data/naf23.pdf'

## initialize agents, and test them on a simple question

### one big prompt agent

In [3]:
agent_one_big_prompt=BigPromptAgent(pdf_path=pdf_path)
query="what is National Ataxia Foundation's total revenue in 2023?"


In [8]:
agent_one_big_prompt=BigPromptAgent(pdf_path=pdf_path)
answer = agent_one_big_prompt.run(query=query)
print(answer)

The National Ataxia Foundation's total revenue in 2023 was $4,184,787. This is the sum of total support ($3,205,136) and total revenue ($979,651), as shown on page 7 of the provided financial report.


### ReAct agent with RAG

In [12]:
prefix='react_data/output/470041917927123113/0'
with open('../config.json', 'r') as file:
    ids = json.load(file)
bucket_name = ids['bucket_name']

chunk_json_list=read_jsonl_to_json_list(bucket_name, prefix)
text_chunks=[]
for json_obj in chunk_json_list:
    text_chunks.append(json_obj['content'])
    
data_path='../data/naf23.pdf'
rag_instance=RAG(pdf_path= data_path, chunking_method='N/A', input_chunks=text_chunks)


current file name: react_data/output/470041917927123113/0/naf23-0.jsonl


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [28]:
def search1(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base, you can use this search function to search for financial information of specific companies you do not know, treat this function as a internal wiki search that you can use

    Args:
        query (str): input query            
    """
    results = rag_instance.search(query, method='ensemble', num_top_chunks=2)
    return json.dumps({"relevant information in the auditor notes": results})


# convert this search function to a Tool object
search_tool_rag_chunks = convert_to_tool(search1)

In [29]:
agent_react = ReactAgent(tools=[search_tool_rag_chunks])

In [32]:
agent_react.run(query="what is the total revenue of National Ataxia Foundation in 2023, not including support?")

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "# National Ataxia Foundation\n\nStatements of Activities (Continued) For the Years Ended December 31, 2023 and 2022\n\n|-|-|-|-|\n|  |  | 2022 |  |\n|  | Without Donor Restriction | With Donor Restriction | Total |\n| Support and Revenue Support |  |  |  |\n| Contributions, memorials and honorariums In-kind contributions | $ 1,638,224 9,870 | $ 1,767,801 | $ 3,406,025 9,870 |\n| Total Support | 1,648,094 | 1,767,801 | 3,415,895 |\n| Revenue |  |  |  |\n| Conference income | 309,574 |  | 309,574 |\n| Earned income | 800,000 |  | 800,000 |\n| Investment loss | (127,884) |  | (127,884) |\n| Total Revenue | 981,690 |  | 981,690 |\n| Net Assets Released from Restrictions | 1,278,770 | (1,278,770) |  |\n| Total Support and Revenue | 3,908,554 | 489,031 | 4,397,585 |\n| Expenses |  |  |  |\n| Program services |  |  |  |\n| Research | 1,837,520 |  | 1,837,520 |\n| Education and service | 579,104 |  | 579,104 |\n| Drug Development Collaborative | 9

'The total revenue for the National Ataxia Foundation in 2023, not including support, was $981,690.'

### ReAct agent that returns the full pdf

In [16]:
auditor_notes_doc=None
with open(pdf_path, 'rb') as fp:
    auditor_notes_doc=Part.from_data(data=fp.read(),mime_type='application/pdf')

In [39]:
# define a search function that returns the entire pdf read from Part.from_data()
def search2(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base, you can use this search function to search for financial information of specific companies you do not know, treat this function as a internal wiki search that you can use

    Args:
        query (str): input query            
    """
    return auditor_notes_doc

# convert this search function to a Tool object
search_tool_rag_full_pdf = convert_to_tool(search2)

In [72]:
agent_react_full_pdf = ReactAgent(tools=[search_tool_rag_full_pdf])

In [41]:
# make sure they have different tools
print(agent_react.tools)
print(agent_react_full_pdf.tools)

In [31]:
agent_react_full_pdf.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 1  ----------------------------------------


"The National Ataxia Foundation's total support and revenue for the year ended December 31, 2023, was $4,184,787."

### question 1

In [82]:
query_short="Cash and cash equivalents decreased from $1,969,164 in 2022 to $1,008,716 in 2023 for National Ataxia Foundation, what are the potential drivers"

In [83]:
rich_Markdown(agent_one_big_prompt.run(query=query_short))

The Statement of Cash Flows shows a decrease in cash and cash equivalents of $960,448.  This was driven by a net   
cash outflow from operating activities of $955,522 and a net cash outflow from investing activities of $4,926.  The
net cash outflow from operating activities was primarily due to a decrease in net assets of $811,671.  The net cash
outflow from investing activities was due to the purchase and sale of investments.

In [84]:
agent_react.run(query=query_short)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "appropriate in the circumstances, but not for the purpose of expressing an opinion on the effectiveness of the  \nFoundation\u2019s  internal control. Accordingly, no such opinion is expressed.  \n \n\u2022 Evaluate the appropriateness of accounting policies used and the reasonableness of significant accounting \nestimates made by management, as well as evaluate the overall presentation of the financial statements.  \n \n\u2022 Conclude whether, in our judgment, there are conditions or events, considered in the aggregate, that raise \nsubstantial doubt about the Foundation\u2019s  ability to continue as a going concern for a reasonable period of time.  \n \nWe are required to communicate with those charged with governance regarding, among other matters, the planned scope \nand timing of the audit, significant audit findings, and certain internal control related matters that we identified during t he \naudit.  \n \nAbdo  \nMinneapolis, Minn

{"relevant information in the auditor notes": "Annual Financial  \nReport  \nNational Ataxia Foundation   \nSt. Louis Park, Minnesota  \n \n \nFor the years ended December 31, 2023  and 2022  \n National Ataxia Foundation  \nTable of Contents  \nDecember 31, 2023  and 2022  \nPage No.  \nIndependent Auditor's Report  3 \nFinancial Statements  \nStatement s of Financial Position  6 \nStatement s of Activities  7 \nStatements of Functional Expenses  9 \nStateme nts of Cash Flows  11 \nNotes to the Financial Statements  12 \n2 \n  \n \n \n \n \n \nINDEPENDENT AUDITOR'S REPORT  \n \n \nBoard of Directors  \nNational Ataxia Foundation  \nSt. Louis Park , Minnesota  \n \nOpinion  \n \nWe have audited the accompanying financial statements of National Ataxia Foundation  (the Foundation ), which comprise \nthe statements of financial position as of December 31, 2023  and 2022 , and the related statements of activities, \nfunctional expenses and cash flows for the years then ended, and the relat

{"relevant information in the auditor notes": "Annual Financial  \nReport  \nNational Ataxia Foundation   \nSt. Louis Park, Minnesota  \n \n \nFor the years ended December 31, 2023  and 2022  \n National Ataxia Foundation  \nTable of Contents  \nDecember 31, 2023  and 2022  \nPage No.  \nIndependent Auditor's Report  3 \nFinancial Statements  \nStatement s of Financial Position  6 \nStatement s of Activities  7 \nStatements of Functional Expenses  9 \nStateme nts of Cash Flows  11 \nNotes to the Financial Statements  12 \n2 \n  \n \n \n \n \n \nINDEPENDENT AUDITOR'S REPORT  \n \n \nBoard of Directors  \nNational Ataxia Foundation  \nSt. Louis Park , Minnesota  \n \nOpinion  \n \nWe have audited the accompanying financial statements of National Ataxia Foundation  (the Foundation ), which comprise \nthe statements of financial position as of December 31, 2023  and 2022 , and the related statements of activities, \nfunctional expenses and cash flows for the years then ended, and the relat

'The decrease in cash and cash equivalents for the National Ataxia Foundation from 2022 to 2023 is primarily attributed to a negative cash flow from operating activities ($955,522). This indicates the foundation spent more than it brought in during 2023.  A significant realized and unrealized loss on investments further exacerbated the decrease.  While there was a small negative cash flow from investing activities, the operating loss was the primary driver.'

In [85]:
agent_react_full_pdf.run(query=query_short)

---------------------------------------- iteration 0  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 1  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 2  ----------------------------------------


"The National Ataxia Foundation's decrease in cash and cash equivalents from $1,969,164 in 2022 to $1,008,716 in 2023 was primarily due to an increase in program spending, outpacing the available revenue and resulting in negative cash flow from operations.  The largest increases in program spending were related to research, education and services, and the Drug Development Collaborative."

* <span style="color: blue; font-weight: bold;"> all methods fail to go to check `"Note 10: Liquidity and Availability of Financial Assets"` </span>
* <span style="color: blue; font-weight: bold;"> especially for RAG, the LLM wounld not serach for 'liquidity', so it never goes to that note </span>
* <span style="color: blue; font-weight: bold;"> **the ReAct with full pdf even breaks our system prompt's specification by outputing `<observation>`** </span>

### question 2

In [54]:
query_short="Why did deferred revenue increase significantly in 2023 for National Ataxia Foundation?"

In [55]:
agent_one_big_prompt.run(query=query_short)

"The auditor's report states that deferred revenue consists of payments received in advance that relate to the Collaborative and conferences. All deferred revenue will be recognized over the next year. Deferred conference revenue was $9,810 in 2023 and $22,290 in 2022. Deferred Drug Development Collaborative revenue was $100,000 in 2023 and $0 in 2022.\n\nTherefore, the significant increase in deferred revenue in 2023 is due to the $100,000 in deferred Drug Development Collaborative revenue received in advance."

In [56]:
agent_react.run(query=query_short)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "the years ended December 31, 2023  and 2022 , respectively. Deferred Drug D evelopment Collaborative revenue \nwas $100,000  and $0 for the years ended December 31, 2023  and 2022 , respectively.  \n \nThe Foundation \u2019s other revenues are explicitly excluded from the scope of ASC Topic 606 and are not recorded in \naccordance with that standard.  All contributions are considered to be available for unrestricted use unless specifically \nrestricted by the donor. The Board\u2019s general policy is to allocate up to thirty percent of all restricted contributions for \nunrestricted use unless the contribution was solicited by the Foundation for a sp ecific purpose.  The board has discretion \nto use this allocation or waive it.  \n15 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 1:  Summary of Significant Accounting Policies (Continued)  \n \nJ. Property and Equipment  \n \nPr

'The significant increase in deferred revenue for the National Ataxia Foundation in 2023 is primarily attributable to a rise in deferred Drug Development Collaborative revenue, from $0 in 2022 to $100,000 in 2023. This reflects a substantial increase in funding received in advance for collaborative drug development projects, which will be recognized as revenue over the duration of the collaborations.'

* <span style="color: blue; font-weight: bold;"> both correct </span>
* <span style="color: blue; font-weight: bold;"> ReAct just use 1 iteration </span>

### question 3

In [57]:
query="What contributed to the significant increase in net assets with donor restrictions from 2022 to 2023 for National Ataxia Foundation"

In [58]:
agent_one_big_prompt.run(query=query)

"The provided auditor's report does not contain information regarding the drivers of the increase in net assets with donor restrictions from 2022 to 2023.  Therefore, I cannot answer your question.  The Statement of Activities shows that net assets with donor restrictions increased by $638,761, but the drivers of this change are not detailed in the provided notes."

In [73]:
agent_react.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "and stored at the NINDS SCA -BRAC biorepository. Both de-identified clinical data and biofluids are available \nfor request from non -participating investigators for approved research projects.  \n  \n13 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 1:  Summary of Significant Accounting Policies (Continued)  \n \nB. Basis of Accounting and Presentation  \n \nThe accompanying financial statements have been prepared using  the accrual basis of accounting in accordance with \naccounting principles generally accepted in the United States of America.   \n \nRevenues are recorded when earned and expenses are recorded when a liability is incurred. Contributions received are \nrecorded as an increase in non -donor -restricted or donor -restricted support depending on the existence or nature of any \ndonor restrictions.  Accordingly, net assets of the Foundation and changes therein are 

{"relevant information in the auditor notes": "and stored at the NINDS SCA -BRAC biorepository. Both de-identified clinical data and biofluids are available \nfor request from non -participating investigators for approved research projects.  \n  \n13 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 1:  Summary of Significant Accounting Policies (Continued)  \n \nB. Basis of Accounting and Presentation  \n \nThe accompanying financial statements have been prepared using  the accrual basis of accounting in accordance with \naccounting principles generally accepted in the United States of America.   \n \nRevenues are recorded when earned and expenses are recorded when a liability is incurred. Contributions received are \nrecorded as an increase in non -donor -restricted or donor -restricted support depending on the existence or nature of any \ndonor restrictions.  Accordingly, net assets of the Foundation and changes therein are 

{"relevant information in the auditor notes": "and stored at the NINDS SCA -BRAC biorepository. Both de-identified clinical data and biofluids are available \nfor request from non -participating investigators for approved research projects.  \n  \n13 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 1:  Summary of Significant Accounting Policies (Continued)  \n \nB. Basis of Accounting and Presentation  \n \nThe accompanying financial statements have been prepared using  the accrual basis of accounting in accordance with \naccounting principles generally accepted in the United States of America.   \n \nRevenues are recorded when earned and expenses are recorded when a liability is incurred. Contributions received are \nrecorded as an increase in non -donor -restricted or donor -restricted support depending on the existence or nature of any \ndonor restrictions.  Accordingly, net assets of the Foundation and changes therein are 

"Based on the available financial statements, the National Ataxia Foundation's net assets with donor restrictions increased by $638,761 from $2,041,806 in 2022 to $2,680,567 in 2023.  While the statements confirm this increase and show that $1,614,715 in contributions were received with donor restrictions in 2023, they do not provide a specific breakdown of the reasons for the change.  Further information, such as the organization's annual report or a more detailed financial review, would be needed to determine the specific factors contributing to this increase."

In [75]:
agent_react_full_pdf.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 1  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 2  ----------------------------------------


"The National Ataxia Foundation's net assets with donor restrictions increased by $638,761 from 2022 to 2023. This was primarily due to contributions of $1,614,715 with donor restrictions, offset by $975,954 released from restrictions during 2023.  Several specific restricted funds saw increases, as detailed in Note 6 of the financial statements."

* <span style="color: blue; font-weight: bold;"> big prompt cannot find the answer </span>
* <span style="color: blue; font-weight: bold;"> ReAct with RAG cannot, becuase RAG keeps returning the incorrect chunks </span>
* <span style="color: blue; font-weight: bold;"> ReAct with full pdf is partially correct, `"as detailed in Note 6 of the financial statements."` </span>

### question 4

In [87]:
query="What caused the significant decrease in the long-term operating lease liability from 2022 to 2023 for National Ataxia Foundation?"


In [88]:
agent_one_big_prompt.run(query=query)

'This document does not contain information regarding operating leases or the reason for the decrease in long-term operating lease liability.  Therefore, I cannot answer your question.'

In [90]:
agent_react.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "according to the Foundation 's elected policy. The Foundation 's lease agreement does not contain any material residual \nvalue guarantees or material restrictive covenants.  \n \nAdditional information about the Foundation \u2019s lease for the year ended December 31, 2023 , is as follows:  \n \nLease expense (included in operating expenses)\nOperating lease expense 33,406 $          \nVariable lease expense 27,997             \nTotal Lease Expense: 61,403 $          \nOther Information\nCash paid for amounts included in the measurement of lease liabilities\nOperating cash flows from operating leases 33,240 $          \nWeighted-average remaining lease term in years for operating leases 1.75                 \nWeighted-average discount rate for operating leases 2.333%\nFuture minimum payments for leases are as follows:\nYear Ended December 31, Amount\n2024 33,693 $          \n2025 25,552             \nTotal undiscounted cash flows 59,245  

{"relevant information in the auditor notes": "according to the Foundation 's elected policy. The Foundation 's lease agreement does not contain any material residual \nvalue guarantees or material restrictive covenants.  \n \nAdditional information about the Foundation \u2019s lease for the year ended December 31, 2023 , is as follows:  \n \nLease expense (included in operating expenses)\nOperating lease expense 33,406 $          \nVariable lease expense 27,997             \nTotal Lease Expense: 61,403 $          \nOther Information\nCash paid for amounts included in the measurement of lease liabilities\nOperating cash flows from operating leases 33,240 $          \nWeighted-average remaining lease term in years for operating leases 1.75                 \nWeighted-average discount rate for operating leases 2.333%\nFuture minimum payments for leases are as follows:\nYear Ended December 31, Amount\n2024 33,693 $          \n2025 25,552             \nTotal undiscounted cash flows 59,245  

{"relevant information in the auditor notes": "according to the Foundation 's elected policy. The Foundation 's lease agreement does not contain any material residual \nvalue guarantees or material restrictive covenants.  \n \nAdditional information about the Foundation \u2019s lease for the year ended December 31, 2023 , is as follows:  \n \nLease expense (included in operating expenses)\nOperating lease expense 33,406 $          \nVariable lease expense 27,997             \nTotal Lease Expense: 61,403 $          \nOther Information\nCash paid for amounts included in the measurement of lease liabilities\nOperating cash flows from operating leases 33,240 $          \nWeighted-average remaining lease term in years for operating leases 1.75                 \nWeighted-average discount rate for operating leases 2.333%\nFuture minimum payments for leases are as follows:\nYear Ended December 31, Amount\n2024 33,693 $          \n2025 25,552             \nTotal undiscounted cash flows 59,245  

"I cannot determine the specific cause of the decrease in the National Ataxia Foundation's long-term operating lease liability from 2022 to 2023 with the information currently available to me.  Further research using their full financial statements would be required."

In [93]:
agent_react_full_pdf.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 1  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 2  ----------------------------------------


'The decrease in the long-term operating lease liability from 2022 to 2023 is due to lease payments made by the National Ataxia Foundation during 2023.  The lease is for their office space and expires in September 2025.'

* <span style="color: blue; font-weight: bold;">one big prompt cannot answer</span>
* <span style="color: blue; font-weight: bold;">ReAct with RAG got it wrong, because RAG returns the incorrect chunks. The LLM cannot even distinguish whether the number is 2022 or 2023. For example: `<thought>The provided information shows the lease liability at the end of 2023 ($58,103) and details`. This $58,103 is actually the number for 2022.</span>
* <span style="color: blue; font-weight: bold;">ReAct with full PDF got it right.</span>


# iccrom.pdf

In [97]:
pdf_path='../data/stc.pdf'

In [102]:
# -------------------------------one big prompt agent --------------------------------
agent_one_big_prompt=BigPromptAgent(pdf_path=pdf_path)

# -------------------------------ReAct agent with RAG --------------------------------
rag_instance=RAG(pdf_path= pdf_path, chunking_method='recursive')
def search1(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base, you can use this search function to search for financial information of specific companies you do not know, treat this function as a internal wiki search that you can use

    Args:
        query (str): input query            
    """
    results = rag_instance.search(query, method='ensemble')
    return json.dumps({"relevant information in the auditor notes": results})

search_tool_rag_chunks = convert_to_tool(search1)
agent_react = ReactAgent(tools=[search_tool_rag_chunks])


# -------------------------------ReAct agent with full pdf ---------------------------
auditor_notes_doc=None
with open(pdf_path, 'rb') as fp:
    auditor_notes_doc=Part.from_data(data=fp.read(),mime_type='application/pdf')
    
def search2(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base, you can use this search function to search for financial information of specific companies you do not know, treat this function as a internal wiki search that you can use

    Args:
        query (str): input query            
    """
    return auditor_notes_doc

search_tool_rag_full_pdf = convert_to_tool(search2)
agent_react_full_pdf = ReactAgent(tools=[search_tool_rag_full_pdf])

### question 1

In [105]:
query='Assets changed from €3,448,770.20 in 2020 to €1,526,209.73 in 2021 for Tansport Community. What are the drivers behind this change?'

In [106]:
agent_one_big_prompt.run(query=query)

'Total assets decreased from €3,448,770.20 in 2020 to €1,526,209.73 in 2021. This was driven by a decrease in current assets, specifically cash and cash equivalents, which decreased from €3,254,771.91 to €1,492,643.83.  While there was a slight increase in non-current assets from €8,244.43 to €32,101.43, this was significantly less than the decrease in current assets.  The notes do not provide further details on the drivers of the change in cash and cash equivalents.'

In [107]:
agent_react.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "Net carrying amount at 31 December current year 4,985,03 8,244.40\nNet carrying amount at 31 December prior year 8,244,40 18,045. 67\nDuring 2021 additions made by the Transport Community amounted to EUR 582.25 (for 2020: EUR 3,298.51). Total\namount of depreciation for 2021 is EUR 3,841.65 (For 2020: EUR 13,100.20)PERMANENT SECRETARIAT OF THE TRANSPORT COMMUNITY\nNotes to the Financial Statements -\nFinancial Year 2021\nNote 5: Account payables\nTrade payables\nStaff payables\nLegal commitments\nAccruals\nTotal31 December 2021\nInEUR31 December 2020\nIn EUR\n42,986. 83\n14,940. 43\n690, 745.00\n7,750. 0019,488.73\n1,144. 58\n162, 260. 00\n3,000. 00\n756,422. 26 185,893.31\nTrade payable include invoices received from suppliers for goods and services received by 31 December 2021 but not\nyet settled at the end of the year. Trade payables in total amout of EUR 42,986.83 are related to office running costs,\ntravel costs, event participation

{"relevant information in the auditor notes": "Usage of obligation dedicated to assets\nDecrease in receivables\n(Increase )/Decrease in prepayments\nlncrease/(Decrease) in trade payables and other liabilities\nIncrease in legal commitments\nlncrease/(Decrease) in unused appropriations\nNet cash flows from operating activities\nCASH FLOWS FROM INVESTING ACTIVITIES\nAdditions of non-current assets\nNet cash flows from investing activities\nCASH FLOWS FROM FINANCING ACTIVITIES\nGrant contribution for assets\nNet cash flows from financing activities\nNet increase/(decrease) in cash and cash equivalents\nCash and cash equivalents at beginning of period\nCash and cash equivalents at end of period13,699. 88\n(13,699.88)\n185,753. 87\n(1,464.48)\n41,989. 71\n528, 485. 00\n(2,516,892. 17)\n(1,762,128.07)\n(37,556. 88)\n(37,556.88)\n37,556. 88\n37,556.88\n(1,762,128.07)\n3,254, 771. 91\n1,492,643.8316,136.92\n(16,136.92)\n927, 973. 38\n648. 13\n(11,076.60)\n106, 943.28\n2,076,328.05\n3,100,816.

{"relevant information in the auditor notes": "carrying value of the tangible assets is also shown as an investment donations in the amount of EUR32, 101.43\nThe carrying amount as at 31 December 2021 of investments consists of the carrying amount as at January 2021\namounting to EUR 8,244.43 and additions amounting to EUR 37,556.88 that were reduced by depreciation and\namortization of EUR 13,699.88 The additions of 2021 include purchases of Enterprise Resource Planning software,\nother software licence and office equipment.PERMANENT SECRETARIAT OF THE TRANSPORT COMMUNITY\nNotes to the Financial Statements -\nFinancial Year 2021\nNote 8: Revenue\nContributions\nBudget contributions parties\nIn kind contribution from EU\nAllocation to investments\nLegal commitments, prior year\nLegal commitments, current year\nUnused appropriations\nTotal contributions:2021\nIn EUR\n3,000, 000. 00\n(37,556.88)\n162, 260.00\n(690, 745. 00)\n(688, 993.55)\n1,744,964.572020\nIn EUR\n3,000, 000. 00\n10,991

"The decrease in Transport Community's assets from €3,448,770.20 in 2020 to €1,526,209.73 in 2021 is primarily due to a significant increase in budget utilization. While the budgeted contributions remained consistent at €3,000,000.00 for both years, a much larger portion of the budget was spent in 2021 compared to 2020, resulting in a decrease in cash and cash equivalents and overall assets.  Specifically, unused budget appropriations decreased from €2,076,328.05 in 2020 to €688,993.55 in 2021. This increased spending, coupled with the timing of budget contributions received (reflected in the decrease in receivables), explains the significant drop in assets."

In [108]:
agent_react_full_pdf.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 1  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 2  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 3  ----------------------------------------


"The decrease in Transport Community's assets from €3,448,770.20 in 2020 to €1,526,209.73 in 2021 is primarily driven by a significant decrease in cash and cash equivalents. This decrease is largely attributed to a substantial increase in unused budget appropriations, totaling €737,686.05 in 2021.  A large portion of these unused appropriations (€688,993.55) are from the 2021 fiscal year and are proposed for repayment to contributing parties, mainly the European Union.  Additional contributing factors include a decrease in receivables and prepayments. While the precise reasons for the under-spending are not detailed in the financial statements, it is likely due to project delays, administrative hurdles, or other operational factors impacting the execution of the planned budget.  Further investigation would be needed to determine the specific causes of the under-spending."

* <span style="color: blue; font-weight: bold;"> one big prompt cannot answer<span>
* <span style="color: blue; font-weight: bold;">ReAct with RAG does find the reasons: decreased cash and decreased unused appropriations <span>
* <span style="color: blue; font-weight: bold;">ReAct with full pdf finds the reasons, but they are phrased incorrectly `This decrease is largely attributed to a substantial increase in unused budget appropriations, totaling €737,686.05 in 2021` the unused budget appropriations does not increase in 2021, it decreases<span>
* <span style="color: blue; font-weight: bold;">the reason found by chatgpt does not seem to be correct</span>

### question 2

In [109]:
query="Total expenses increased significantly in 2021 compared to 2020. What are the main reasons behind this increase?"


In [110]:
agent_one_big_prompt.run(query=query)

'Total expenses in 2021 amounted to €1,744,964.57, compared to €821,384.63 in 2020. The main drivers for this increase are:\n\n* **Staff costs:** Increased from €710,717.99 in 2020 to €1,385,839.57 in 2021. This is mainly due to the fact that the Permanent Secretariat was fully staffed with 21 employed officials in 2021 (with the exception of January), compared to partial staffing in 2020.\n* **Operating expenses:** Increased from €110,666.64 in 2020 to €359,125.00 in 2021. This is mainly due to higher running costs (€187,678.09 in 2021 vs. €105,572.94 in 2020), which include travel costs, office costs, equipment and software, other costs and services, and recruitment costs.  The notes mention that travel costs were lower than budgeted due to the pandemic and the use of private vehicles for regional travel. However, the overall increase in operating expenses suggests that other cost categories increased significantly.  The notes also mention that studies and technical assistance costs 

In [111]:
agent_react.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "Budget contributions parties\nIn kind contribution from ED\nLegal commitments, prior year\nLegal commitments, current year\nAllocation to investments\nUnused appropriations\nTotal contributionsNotes\n831 December 2021\n(in EUR)\n3,000, 000. 00\n162,260. 00\n(690, 745.00)\n(37,556.88)\n(688, 993.55)\n1,744, 964. 5731 December 2020\n(in EUR)\n3,000, 000. 00\n10,991. 18\n55,316. 72\n(162, 260.00)\n(6,335.22)\n(2,076,328.05)\n821,384.63\nTOTAL REVENUE 1,744, 964. 57 821, 384. 63\nEXPENSES\nStaff costs\nStaff salaries\nHealth/Pension/Social security contr.\nOther staff costs\nTotal staff costs\nDepreciation and Amortization Expense\nFixed assets amortization charge\nDepreciation and amortization usage\nTotal fixed assets amortization\nOperating expenses\nRunning costs\nRecruitment costs\nFinance costs\nTotal operating expenses1,181,775. 05\n113, 169. 16\n90,895. 36\n1,385, 839. 57\n13,699. 88\n(13,699. 88)\n350, 390. 24\n8,734.76660, 652. 94\n4

"The increase in total expenses in 2021 compared to 2020 for the Permanent Secretariat of the Transport Community is primarily due to increased staff costs (salaries, benefits, and other staff costs), higher operating expenses (running costs, recruitment, and finance costs), and potentially higher legal commitments.  The provided data doesn't offer a detailed breakdown of the specific drivers within each category, but it suggests these are the main contributing factors."

In [112]:
agent_react_full_pdf.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 1  ----------------------------------------


"The financial statements reveal a significant increase in total expenses in 2021 (EUR 1,744,964.57) compared to 2020 (EUR 821,384.63).  Key drivers include:\n\n* **Staff Costs:** A substantial rise from EUR 710,717.99 in 2020 to EUR 1,385,839.57 in 2021. This is primarily due to increased staffing, with the Secretariat being fully staffed with 21 officials for most of 2021, compared to partial staffing in 2020.  Increased salaries and benefits contribute to this rise.\n* **Operating Expenses:**  Rose from EUR 110,666.64 to EUR 359,125.00. This includes a significant increase in running costs (travel, office expenses, etc.) likely due to increased activity and potentially higher prices.  Recruitment costs also contributed to the increase in 2021.\n* **Legal Commitments:**  A large portion of the 2021 expenses (EUR 690,745.00) relates to contractual commitments for various projects and services, including development of observatories, IT systems, and technical assistance. These commitme

* <span style="color: blue; font-weight: bold;"> all three methods find the answer because it is very straigtforward<span>


### question 3

In [113]:
query="Unused budget appropriations decreased from €3,254,632.46 in 2020 to €737,686.05 in 2021. What caused this decrease?"

In [114]:
agent_one_big_prompt.run(query=query)

'Unused budget appropriations for 2019 and 2020 were repaid in 2021, resulting in a decrease in unused appropriations from €3,254,632.46 in 2020 to €737,686.05 in 2021.  Note 6 explains that unused appropriations are proposed for repayment to the contracting parties, and Note 13 shows the unused appropriations for 2019 and 2020 were repaid in 2021. Note 13 also shows that €688,993.55 of the 2021 budget was unused.'

In [115]:
agent_react.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "public information campaign, consultancy services, technical assistance for e-freight deployment, technical assistance\nfor e-tolling interoperability, services for development of a website, capacitby building for transport facilitation.\nThe accruals related to the costs for external audit of 2021 Financial Statements.\nNote 6: Unused appropriations\nUnused appropriations (budget parties)31 December 2021 31 December 2020\nIn EUR InEUR\n737,686.05 3,254, 632.46\nUnused budget appropriations (not used and not committed at the end of the financial year) are shown as liabilities\ntowards the Contracting Parties. Total amount as at 31 Decemeber 2021 is EUR 737,686.05 representing unused\nappropriations from the financial year 2019 and 2020 of EUR 48,692.50 and EUR 688,993.55 from Fiscal Year 2021.\nThe Budget Committee has recommended that the unused appropriations belonging to the South East Europe parties\nto be carried forward as Parties' c

'The decrease in unused budget appropriations from €3,254,632.46 in 2020 to €737,686.05 in 2021 is due to a combination of factors, including the utilization of funds for various projects and activities such as public information campaigns, consultancy services, technical assistance for e-freight deployment and e-tolling interoperability, services for website development, and capacity building for transport facilitation.  Additionally, the policy of carrying forward unused appropriations belonging to South East Europe parties and repaying unused appropriations belonging to the European Union contributed to the decrease.  The unused amount in 2021 consists of €48,692.50 from 2019, €688,993.55 from 2020, indicating that a significant portion of the 2020 unused appropriations were either spent or reallocated in 2021.'

In [117]:
agent_react_full_pdf.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------


observation is non-printable Part object, probably the full pdf
---------------------------------------- iteration 1  ----------------------------------------


'The decrease in unused budget appropriations from €3,254,632.46 in 2020 to €737,686.05 in 2021 is shown in the Statement of Financial Position.  The notes to the financial statements provide further details, indicating that the unused amount consists of unused appropriations from 2019 and 2020 (€48,692.50) and unused appropriations from 2021 (€688,993.55). The notes also explain that unused appropriations are proposed for repayment to the contributing parties based on a methodology outlined in the budget report.  The decrease reflects a significant improvement in budget utilization in 2021 compared to the prior years.'

* <span style="color: blue; font-weight: bold;"> the information is in Note 5, one big prompt agent failed to find that <span>
* <span style="color: blue; font-weight: bold;"> ReAct with RAG correctly found it<span>
* <span style="color: blue; font-weight: bold;"> ReAct with full pdf also did not find the specific reason<span>